In [ ]:
import xarray as xr
import numpy as np
import sys
#import hydra
from typing import Optional
import datetime


ATTRS ={'time': {'dtype': 'int64',}, # 'units': 'days since 2019-01-01', }, 
        'lon': {'dtype': 'float32', 'valid_min': -180., 'valid_max': 180.,
                'long_name': 'longitude', 'standard_name': 'longitude',
                'units': 'degrees_east'},
        'lat': {'dtype': 'float32', 'valid_min': -90., 'valid_max': 90.,
                'long_name': 'latitude', 'standard_name': 'latitude',
                'units': 'degrees_north'},
        'ugos': {'dtype': 'float32', 'valid_min': -100., 'valid_max': 100.,
                'units': 'm/s',
                'long_name': 'Eastward total velocity',
                },
        'vgos': {'dtype': 'float32', 'valid_min': -100., 'valid_max': 100.,
                'units': 'm/s',
                'long_name': 'Northward total velocity',
                },
                }
COMPLEVEL = 4
ZLIB = True
FV32 = 1.e+20
ENC_FV = {'zlib': ZLIB, 'complevel': COMPLEVEL, '_FillValue': FV32,
          'dtype': 'float32'}
ENC = {'zlib': ZLIB, 'complevel': COMPLEVEL, 'dtype': 'float32'}


def save(ds: xr.Dataset, listkey: list, file_out: str, 
         ATTR_VARS: Optional[dict] = ATTRS, fillvalue: Optional[float] = FV32):
    encoding = {}
    for key in listkey:
        print(key)
        dic_attr = {}
        if key in ATTR_VARS.keys():
            dic_attr = ATTR_VARS[key]
        ds[key].values[np.isnan(ds[key].values)] = fillvalue
        ds = ds.assign({key: (['time', 'lat', 'lon'], ds[key].values,
                            dic_attr)})

        encoding[key] = ENC_FV
        print(ENC_FV)
    for key in ('lon', 'lat', 'time'):
    #for key in ('lon', 'lat'):
        dic_attr = {}
        if key in ATTR_VARS.keys():
            dic_attr = ATTR_VARS[key]
        ds = ds.assign({key: ([key,], ds[key].values,
                            dic_attr)})
        encoding[key] = ENC
    encoding['time']['dtype'] = 'int64'
    encoding['lat']['dtype'] = 'float32'
    encoding['lon']['dtype'] = 'float32'
    
    print(encoding)

    t0 = ds['time'].values[0] - np.timedelta64(12, 'h')
    t1 = ds['time'].values[-1] + np.timedelta64(12, 'h')

    ds.attrs['title'] = 'multivar_uv'
    ds.attrs['project'] = 'multivar_uv'
    ds.attrs['summary'] = ''
    ds.attrs['references'] = ''
    ds.attrs['institution'] = 'IMT Atlantic'
    ds.attrs['creator_name'] = 'Theo'
    ds.attrs['version_id'] = '0.1'
    ds.attrs['date_created'] = str(datetime.datetime.now())
    ds.attrs['geospatial_lat_min'] = f'{np.min(ds["lat"].values)}N'
    ds.attrs['geospatial_lat_max'] = f'{np.max(ds["lat"].values)}N'
    ds.attrs['geospatial_lon_min'] = f'{np.min(ds["lon"].values)}E'
    ds.attrs['geospatial_lon_max'] = f'{np.max(ds["lon"].values)}E'
    ds.attrs['time_coverage_start'] = np.datetime_as_string(t0, unit='s')
    ds.attrs['time_coverage_end'] = np.datetime_as_string(t1, unit='s')
    #ds.to_netcdf(file_out, 'w', format="NETCDF4", encoding=encoding)
    return ds,encoding

from glob import glob
import xarray as xr

path_file = "/Odyssey/private/t22picar/multivar_uv/rec/ssh_duacs_sst_w_to_u_v_11d_15m_ageos/daily_ageos/"
list_of_maps = sorted(glob(path_file+'/unet_rec_*.nc'))
ds_maps = xr.open_mfdataset(list_of_maps, combine='nested', concat_dim='time')


folder_data = "/Odyssey/private/t22picar/multivar_uv/rec/duacs_geos/daily/"
list_of_maps = sorted(glob(folder_data+'/unet_rec_*.nc'))
maps_geo = xr.open_mfdataset(list_of_maps, combine='nested', concat_dim='time')

ds_maps.ugos.values = ds_maps.ugos.values + maps_geo.ugos.values
ds_maps.vgos.values = ds_maps.vgos.values + maps_geo.vgos.values

import os
# Chemin du dossier que vous souhaitez créer
dossier_path_daily = f'{path_file}/daily'
# Créer le dossier
os.makedirs(dossier_path_daily, exist_ok=True)


### SAVING ####
# Récupérer la liste des variables sans les dimensions
variables = [var for var in ds_maps.variables if var not in ds_maps.dims]
ds_maps,encoding = save(ds_maps,variables,dossier_path_daily)


#Select day per day 
from datetime import datetime, timedelta
# Définir la date de début et la date de fin
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 1, 1)

# Boucle sur chaque jour de la période
current_date = start_date
while current_date < end_date:
    #print(current_date.strftime('%Y-%m-%d'))  # Affiche la date au format AAAA-MM-JJ
    ds_map_day=ds_maps.sel(time=current_date)
    folder_out = dossier_path_daily+f"/unet_rec_{current_date.strftime('%Y-%m-%d')}.nc"
    ds_map_day.to_netcdf(folder_out, 'w', format="NETCDF4", encoding=encoding)
    current_date += timedelta(days=1)  # Passe au jour suivant



ugos
{'zlib': True, 'complevel': 4, '_FillValue': 1e+20, 'dtype': 'float32'}
vgos
{'zlib': True, 'complevel': 4, '_FillValue': 1e+20, 'dtype': 'float32'}
{'ugos': {'zlib': True, 'complevel': 4, '_FillValue': 1e+20, 'dtype': 'float32'}, 'vgos': {'zlib': True, 'complevel': 4, '_FillValue': 1e+20, 'dtype': 'float32'}, 'lon': {'zlib': True, 'complevel': 4, 'dtype': 'float32'}, 'lat': {'zlib': True, 'complevel': 4, 'dtype': 'float32'}, 'time': {'zlib': True, 'complevel': 4, 'dtype': 'float32'}}
